In [5]:
import {
  ChatAnthropic,
  ChatOpenAI,
  ChatPromptTemplate,
  JsonOutputParser,
  StringOutputParser,
} from "packages/deps.ts";

const openAIApiKey = Deno.env.get("OPENAI_API_KEY") ?? "";
const anthropicApiKey = Deno.env.get("ANTHROPIC_API_KEY") ?? "";



In [21]:
const model = new ChatAnthropic({
  model: "claude-3-5-sonnet-20240620",
  temperature: 0
});

const model2 = new ChatOpenAI({
  model: "gpt-3.5-turbo",
  temperature: 0
});

const properties = {
      anecdotes: { 
          type: "array", 
          items: {
              text: { type: "string", description: "The verbatim transcript of the anecdote" },
              desc: { type: "string", description: "A summary of the anecdote." },
              filename: { type: "string", description: "The name of the file containing the anecdote." },
              topics: { type: "string", description: "A comma-separated list of topics related to the anecdote." },
              rationale: { type: "string", description: "A rationale for the confidence rating." },
              confidence: { type: "number", description: "A floating point confidence rating from 0 to 1, where 0 doesn't relate to the prompt and 1 relates best." }
            }
      }
}
const structuredLlm = model.withStructuredOutput({
  name: "anecdote",
  description: "an anecdote from the text relating to the user prompt.",
  input_schema: {
    type: "object",
    properties,
  }
});

const structuredLlm2 = model2.withStructuredOutput({
  name: "anecdote",
  description: "an anecdote from the text relating to the user prompt.",
  parameters: {
    title: "Anecdote",
    type: "object",
    properties,
  },
});

In [19]:

const document = "Filename: P1008801.MP4 \nContent: What'd you have for breakfast? Perfect. Wonderful. Breakfast of champions. What are cost? Irrigation. They're in the first question. What are cost savings accounts are service. I'm sorry. Just look at Jennifer. I know. Right here businesses thinking about the service or anyone? No, you just. I have two cameras for a reason. That way I can chop you in whenever I need to. Because I don't want to share an example because there's so many little things I would have to explain that would give people. Oh, my God, I'm going to call them because there's all these little things you have to do to get it. Okay, so maybe walk us through what is. I have a great. Whenever you're ready. The way that you can better the savings you expect to see in a cost. Typically, we can take 20% of your public purchase price. Take that as a production. More than you. Walk us through. Like, what should we do, though? Like, what should people expect? Sure. Yeah. So what should people expect when they come to start Costa? What you can expect when you come to meet us for the first time. And after that we'll reach out. Architectural plans of the larger job was good for tax records. Say it again. Where's the initial promise? Walk us through what can people expect? What you can expect in working with us. From there, we'll determine what we will do with that. Properties. They can include cat plans to design our units, closing statements and public. Do you want to talk about any client experience, like tax services? Should we offer our clients or. We're going to need a separate one. Okay. Okay. Do you have anything to say about the IR's? I mean, so if, like you talk about. It's going to be a. Okay. Okay. Yeah. But. The problem there is the word. You could talk through. You know, just like, what are you and your team doing to help make the client experience. Right. One of the things we're doing to make sure. One of the things we're doing to make sure that they're doing so they can have a good experience. Yeah, I like it. I think we're good. I don't have anything else."
const systemMessage = `You have access to a comprehensive database of video transcripts. Your task is to extract all anecdotes from these transcripts based on a user-provided prompt. This task is to be performed using the provided transcript data sections listed below. ${document} `  
const prompt = ChatPromptTemplate.fromMessages([
    ["system", `${systemMessage}`],
    ["user", "{input}"],
  ]);







In [24]:
const chain = prompt.pipe(structuredLlm);
const response = await chain.invoke({ input: "parts where the speaker mentions a name of someone" });
response

{
  anecdotes: [
    {
      filename: "P1008801.MP4",
      text: "Just look at Jennifer. I know. Right here businesses thinking about the service or anyone?",
      desc: "The speaker mentions the name Jennifer while discussing something about businesses and services.",
      confidence: 0.7,
      rationale: "The name Jennifer is clearly mentioned, but the context around it is somewhat unclear. It's not a detailed anecdote, but it does contain a name as requested.",
      topics: "names, Jennifer, business, service"
    }
  ]
}